In [1]:
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import fashion_mnist
from keras.applications import Xception
from tensorflow.keras import layers, models


In [2]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

X_train = np.array([cv2.resize(im, (128, 128)) for im in X_train])
X_test = np.array([cv2.resize(im, (128, 128)) for im in X_test])

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255.0
X_test /= 255.0

X_train = np.stack((X_train,)*3, axis=-1)
X_test = np.stack((X_test,)*3, axis=-1)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [3]:
base_model = Xception(include_top=False, weights='imagenet', input_shape=(128,128,3))
base_model.trainable = False
base_model.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 63, 63, 32)   864         ['input_1[0][0]']                
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 63, 63, 32)  128         ['block1_conv1[0][0]']           
 ation)                                                                                           
                                                                                           

In [4]:
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(10, activation='softmax')(x)

model = models.Model(inputs=base_model.input, outputs=x)

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='accuracy',
    patience=1,
    restore_best_weights=True
)
model.fit(x = X_train, y= y_train, epochs=10, callbacks=[early_stopping])

Epoch 1/10
1875/1875 [==============================] - 1119s 595ms/step - loss: 0.4385 - accuracy: 0.8410
Epoch 2/10
1875/1875 [==============================] - 1114s 594ms/step - loss: 0.3329 - accuracy: 0.8781
Epoch 3/10
1875/1875 [==============================] - 1106s 590ms/step - loss: 0.2902 - accuracy: 0.8919
Epoch 4/10
1875/1875 [==============================] - 1098s 586ms/step - loss: 0.2592 - accuracy: 0.9032
Epoch 5/10
1875/1875 [==============================] - 1078s 575ms/step - loss: 0.2316 - accuracy: 0.9124
Epoch 6/10
1875/1875 [==============================] - 1085s 578ms/step - loss: 0.2050 - accuracy: 0.9227
Epoch 7/10
1875/1875 [==============================] - 1083s 578ms/step - loss: 0.1824 - accuracy: 0.9317
Epoch 8/10
1875/1875 [==============================] - 1080s 576ms/step - loss: 0.1626 - accuracy: 0.9383
Epoch 9/10
1875/1875 [==============================] - 1076s 574ms/step - loss: 0.1445 - accuracy: 0.9452
Epoch 10/10
1875/1875 [==============

In [7]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

313/313 - 181s - loss: 0.4601 - accuracy: 0.8688 - 181s/epoch - 577ms/step

Test accuracy: 0.8687999844551086


In [8]:
model.save('NN/xception_fashion_mnist.h5')